In [32]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


In [33]:
training_data=datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data=datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [34]:
batch_size=256
train_dataloader=DataLoader(training_data,batch_size=batch_size)
test_dataloader=DataLoader(test_data,batch_size=batch_size)
for X,y in test_dataloader:
   print(f"shape of X[N,C,H,W]:{X.shape}")
   print(f"shape of y:{y.dtype}{y.shape}")
   break

shape of X[N,C,H,W]:torch.Size([256, 1, 28, 28])
shape of y:torch.int64torch.Size([256])


In [35]:
device={
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
}
print(f"using {device} device")

using {'cpu'} device


In [36]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten=nn.Flatten()
    self.linear_relu_stack=nn.Sequential(
        nn.Linear(28*28,512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10)

    )
  def forward(self,x):
    x=self.flatten(x)
    logits=self.linear_relu_stack(x)
    return logits


In [37]:
model=NeuralNetwork()
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [38]:
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=1e-3)

In [39]:
def train(dataloader,model,loss_fn,optimizer):
  size=len(dataloader.dataset)
  model.train()
  for batch,(X,y) in enumerate(dataloader):
    pred=model(X)
    loss=loss_fn(pred,y)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if batch%100 ==0:
      loss,current=loss.item(),(batch+1)*len(X)
      print(f"loss:{loss:>7f}[{current:>5d}/{size:>5d}]")



In [40]:
def test(dataloader,model,loss_fn):
 size=len(dataloader.dataset)
 num_batches=len(dataloader)
 model.eval()
 test_loss,correct=0,0
 with torch.no_grad():
  for X,y in dataloader:
    pred=model(X)
    test_loss+=loss_fn(pred,y).item()
    correct+=(pred.argmax(1)==y).type(torch.float).sum().item()
 test_loss/=num_batches
 correct/=size
 print(f"test Erro:\nAcurracy:{(100*correct):>0.1f}%,avg loss:{test_loss:>8f}\n")

In [41]:
epochs=30
for e in range(epochs):
   print(f"Epoch{e+1}\n------------------")
   train(train_dataloader,model,loss_fn,optimizer)
   test(test_dataloader,model,loss_fn)
print("Done!")

Epoch1
------------------
loss:2.300994[  256/60000]
loss:2.285101[25856/60000]
loss:2.274003[51456/60000]
test Erro:
Acurracy:27.2%,avg loss:2.267937

Epoch2
------------------
loss:2.267155[  256/60000]
loss:2.251399[25856/60000]
loss:2.241195[51456/60000]
test Erro:
Acurracy:46.5%,avg loss:2.234108

Epoch3
------------------
loss:2.233275[  256/60000]
loss:2.216192[25856/60000]
loss:2.205403[51456/60000]
test Erro:
Acurracy:48.5%,avg loss:2.196823

Epoch4
------------------
loss:2.195970[  256/60000]
loss:2.176136[25856/60000]
loss:2.163828[51456/60000]
test Erro:
Acurracy:48.2%,avg loss:2.153410

Epoch5
------------------
loss:2.152567[  256/60000]
loss:2.129147[25856/60000]
loss:2.114359[51456/60000]
test Erro:
Acurracy:48.9%,avg loss:2.101718

Epoch6
------------------
loss:2.100977[  256/60000]
loss:2.072863[25856/60000]
loss:2.055089[51456/60000]
test Erro:
Acurracy:50.1%,avg loss:2.039778

Epoch7
------------------
loss:2.039335[  256/60000]
loss:2.005524[25856/60000]
loss:1.9

In [42]:
torch.save(model.state_dict(),"model.pth")
print("saved PyTorch Model State to model.pth")


saved PyTorch Model State to model.pth


In [43]:
model =NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [46]:
classes=[
    "T-shirts/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x,y=test_data[11][0],test_data[11][1]
with torch.no_grad():
  pred=model(x)
  predicted,actual=classes[pred[0].argmax(0)],classes[y]
  print(f'predicted:"{predicted}",Actual:"{actual}"')

predicted:"Sneaker",Actual:"Sandal"
